# Academy of Py
- Average math and reading scores remained relativey static over grade at each school
- Higher per student budget does not have a positive impact on math and reading scores. In fact, two of the highest performing schools spent the least per student.
- School size appears to have an effect on scores. Small and Medium sized schools (less than 2001 students) out-performed larger schools.

In [1]:
import pandas as pd
import numpy as np

In [2]:
schoolFile = "resources/schools_complete.csv"
studentFile = "resources/students_complete.csv"
school = pd.read_csv(schoolFile)
student = pd.read_csv(studentFile)
# rename the column "name" in school df - the "name" column in the student df is the student name
school.rename(columns={"name":"school"}, inplace=True)

##### District Summary

In [3]:
# Total Schools
tSchoolCount = school["school"].nunique()
# Total Students
tStudentCount = student["Student ID"].nunique()
# Total Budget
tBudget = school["budget"].sum()
# Average Math Score
tAvgMath = student["math_score"].mean()
# Average Reading Score
tAvgReading = student["reading_score"].mean()
# % Passing Math
tPassMath = (student[student["math_score"]>=70]["Student ID"].count()/student["Student ID"].count()) * 100
# % Passing Reading
tPassReading = (student[student["reading_score"]>=70]["Student ID"].count()/student["Student ID"].count()) * 100
# Overall Passing Rate (Average of the above two)
# tOverall = np.mean([tPassMath,tPassReading])
tOverall = (student[(student["reading_score"]>=70)&(student["math_score"]>=70)]["Student ID"].count()/student["Student ID"].count()) * 100

keyMetrics = pd.DataFrame({"Total Schools":[tSchoolCount],"Total Students":[tStudentCount],
                           "Total Budget":[tBudget],"Average Math Score":[tAvgMath],
                          "Average Reading Score":[tAvgReading],"% Passing Math":[tPassMath],
                          "% Passing Reading":[tPassReading],"Overall Passing Rate":[tOverall]})
keyMetrics["Total Students"] = keyMetrics["Total Students"]
keyMetrics = keyMetrics[["Total Schools","Total Students","Total Budget","Average Math Score","Average Reading Score","% Passing Math","% Passing Reading","Overall Passing Rate"]]
keyMetrics["Total Students"] = keyMetrics["Total Students"].map("{:,}".format)
keyMetrics["Total Budget"] = keyMetrics["Total Budget"].map("${:,}".format)
keyMetrics["Average Math Score"] = keyMetrics["Average Math Score"].map("{:.0f}%".format)
keyMetrics["Average Reading Score"] = keyMetrics["Average Reading Score"].map("{:.0f}%".format)
keyMetrics["% Passing Math"] = keyMetrics["% Passing Math"].map("{:.0f}%".format)
keyMetrics["% Passing Reading"] = keyMetrics["% Passing Reading"].map("{:.0f}%".format)
keyMetrics["Overall Passing Rate"] = keyMetrics["Overall Passing Rate"].map("{:.0f}%".format)
keyMetrics

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
0,15,"39,170","$24,649,428",79%,82%,75%,86%,65%


##### School Summary

In [4]:
# Create an overview table that summarizes key metrics about each school, including:
# School Name
# School Type
# Total Students
# Total School Budget
schoolSummary = school.copy()
# Per Student Budget
schoolSummary["Budget Per Student"] = schoolSummary["budget"]/schoolSummary["size"]
# Average Math Score
schoolAvgMath = student.groupby("school")["math_score"].mean().to_frame("AvgMath")
# Average Reading Score
schoolAvgReading = student.groupby("school")["reading_score"].mean().to_frame("AvgReading")
# % Passing Math
schoolPassMath = ((student[student["math_score"]>=70].groupby("school")["Student ID"].count()/student.groupby("school")["Student ID"].count())*100).to_frame("%PassMath")
# % Passing Reading
schoolPassReading = ((student[student["reading_score"]>=70].groupby("school")["Student ID"].count()/student.groupby("school")["Student ID"].count())*100).to_frame("%PassReading")
schoolPassOverall = ((student[(student["reading_score"]>=70) & (student["math_score"]>=70)].groupby("school")["Student ID"].count()/student.groupby("school")["Student ID"].count())*100).to_frame("%PassOverall")
# Overall Passing Rate (Average of the above two)
schoolMathReading = schoolPassMath.merge(schoolPassReading, left_index=True, right_index=True).merge(schoolPassOverall, left_index=True, right_index=True)
# schoolMathReading["Overall"] = (schoolMathReading["%PassMath"]+schoolMathReading["%PassReading"])/2
schoolSummaryMerge = schoolSummary.merge(schoolAvgMath, left_on="school", right_index=True).merge(schoolAvgReading, left_on="school", right_index=True).merge(schoolMathReading, left_on="school", right_index=True)
# keep a copy of unformatted data for later manipulation
schoolSummaryFormatted = schoolSummaryMerge.copy()
schoolSummaryFormatted["size"] = schoolSummaryFormatted["size"].map("{:,}".format)
schoolSummaryFormatted["budget"]= schoolSummaryFormatted["budget"].map("${:,}".format)
schoolSummaryFormatted["Budget Per Student"] = schoolSummaryFormatted["Budget Per Student"].map("${:,.0f}".format)
schoolSummaryFormatted["AvgMath"] = schoolSummaryFormatted["AvgMath"].map("{:.0f}".format)
schoolSummaryFormatted["AvgReading"] = schoolSummaryFormatted["AvgReading"].map("{:.0f}".format)
schoolSummaryFormatted["%PassMath"]=schoolSummaryFormatted["%PassMath"].map("{:.0f}".format)
schoolSummaryFormatted["%PassReading"]=schoolSummaryFormatted["%PassReading"].map("{:.0f}".format)
schoolSummaryFormatted["%PassOverall"]=schoolSummaryFormatted["%PassOverall"].map("{:.0f}".format)
schoolSummaryFormatted

,School ID,school,type,size,budget,Budget Per Student,AvgMath,AvgReading,%PassMath,%PassReading,%PassOverall
0,0,Huang High School,District,"2,917","$1,910,635",$655,77,81,66,81,54
1,1,Figueroa High School,District,"2,949","$1,884,411",$639,77,81,66,81,53
2,2,Shelton High School,Charter,"1,761","$1,056,600",$600,83,84,94,96,90
3,3,Hernandez High School,District,"4,635","$3,022,020",$652,77,81,67,81,54
4,4,Griffin High School,Charter,"1,468","$917,500",$625,83,84,93,97,91
5,5,Wilson High School,Charter,"2,283","$1,319,574",$578,83,84,94,97,91
6,6,Cabrera High School,Charter,"1,858","$1,081,356",$582,83,84,94,97,91
7,7,Bailey High School,District,"4,976","$3,124,928",$628,77,81,67,82,55
8,8,Holden High School,Charter,427,"$248,087",$581,84,84,93,96,89
9,9,Pena High School,Charter,962,"$585,858",$609,84,84,95,96,91


##### Top Performing Schools (By Passing Rate)

In [5]:
schoolSummaryFormatted.sort_values(by="%PassOverall", ascending=False).head()

,School ID,school,type,size,budget,Budget Per Student,AvgMath,AvgReading,%PassMath,%PassReading,%PassOverall
4,4,Griffin High School,Charter,"1,468","$917,500",$625,83,84,93,97,91
5,5,Wilson High School,Charter,"2,283","$1,319,574",$578,83,84,94,97,91
6,6,Cabrera High School,Charter,"1,858","$1,081,356",$582,83,84,94,97,91
9,9,Pena High School,Charter,962,"$585,858",$609,84,84,95,96,91
14,14,Thomas High School,Charter,"1,635","$1,043,130",$638,83,84,93,97,91


##### Lowest Performing Schools (By Passing Rate)

In [6]:
schoolSummaryFormatted.sort_values(by="%PassOverall").head()

,School ID,school,type,size,budget,Budget Per Student,AvgMath,AvgReading,%PassMath,%PassReading,%PassOverall
1,1,Figueroa High School,District,"2,949","$1,884,411",$639,77,81,66,81,53
11,11,Rodriguez High School,District,"3,999","$2,547,363",$637,77,81,66,80,53
0,0,Huang High School,District,"2,917","$1,910,635",$655,77,81,66,81,54
3,3,Hernandez High School,District,"4,635","$3,022,020",$652,77,81,67,81,54
12,12,Johnson High School,District,"4,761","$3,094,650",$650,77,81,66,81,54


##### Math Scores by Grade

In [7]:
# Create a table that lists the average Math Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

math = student.groupby(["school","grade"])["math_score"].mean().to_frame("AvgMathScore")
math = pd.pivot_table(math, values="AvgMathScore", index="school",columns=["grade"],aggfunc=np.mean)
math.columns = ["9th","10th","11th","12th"]
math["9th"] = math["9th"].map("{:.0f}".format)
math["10th"] = math["10th"].map("{:.0f}".format)
math["11th"] = math["11th"].map("{:.0f}".format)
math["12th"] = math["12th"].map("{:.0f}".format)
math

,9th,10th,11th,12th
school,,,,
Bailey High School,77,78,76,77
Cabrera High School,83,83,83,83
Figueroa High School,77,77,77,76
Ford High School,78,77,76,77
Griffin High School,84,84,83,82
Hernandez High School,77,77,77,77
Holden High School,83,85,83,84
Huang High School,76,76,77,77
Johnson High School,77,77,77,77


##### Reading Scores by Grade

In [8]:
reading = student.groupby(["school","grade"])["reading_score"].mean().to_frame("AvgReadingScore")
reading = pd.pivot_table(reading, index="school", columns="grade", values="AvgReadingScore")
reading["9th"] = reading["9th"].map("{:.0f}".format)
reading["10th"] = reading["10th"].map("{:.0f}".format)
reading["11th"] = reading["11th"].map("{:.0f}".format)
reading["12th"] = reading["12th"].map("{:.0f}".format)
reading.columns = ["9th","10th","11th","12th"]
reading

,9th,10th,11th,12th
school,,,,
Bailey High School,81,81,81,81
Cabrera High School,84,84,84,84
Figueroa High School,81,81,81,81
Ford High School,81,80,81,81
Griffin High School,84,84,84,83
Hernandez High School,81,81,81,81
Holden High School,83,84,85,84
Huang High School,82,81,80,81
Johnson High School,81,81,81,81


##### Scores by School Spending

In [9]:
# Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
# Average Math Score
# Average Reading Score
# % Passing Math
# % Passing Reading
# Overall Passing Rate (Average of the above two)
#get min and max to determine range
budgetMin = schoolSummaryMerge["Budget Per Student"].min()
budgetMax = schoolSummaryMerge["Budget Per Student"].max()
# budgetMin $578
# budgetMax $655
budgetRange = np.arange(575,700,25)
budgetLabels = []
schoolSummaryMerge["Budget Range"] = pd.cut(schoolSummaryMerge["Budget Per Student"], budgetRange, right=False)
for index, b in enumerate(budgetRange):
    if index != len(budgetRange)-1:
        budgetLabels.append('\$' + str(b) + " - $" + str(b+24))
schoolSummaryMerge["Budget Range Label"] = pd.cut(schoolSummaryMerge["Budget Per Student"],budgetRange, labels=budgetLabels, right=False)
spend = schoolSummaryMerge.merge(student, left_on="school", right_on="school")
# get the columns we need
spendMathReading=spend[["budget","Budget Range Label", "reading_score","math_score"]]

spendMathAvg =spendMathReading.groupby("Budget Range Label")["math_score"].mean().to_frame("MathAvg")
spendReadingAvg = spendMathReading.groupby("Budget Range Label")["reading_score"].mean().to_frame("ReadingAvg")

spendPassMath = ((spendMathReading[spendMathReading["math_score"]>=70].groupby("Budget Range Label")["math_score"].count()/spendMathReading.groupby("Budget Range Label")["math_score"].count())*100).to_frame("% Passing Math")
spendPassReading = ((spendMathReading[spendMathReading["reading_score"]>=70].groupby("Budget Range Label")["reading_score"].count()/spendMathReading.groupby("Budget Range Label")["reading_score"].count()) * 100).to_frame("% Passing Reading")
spendPassOverall = ((spendMathReading[(spendMathReading["math_score"]>=70)&(spendMathReading["reading_score"]>=70)].groupby("Budget Range Label")["budget"].count()/spendMathReading.groupby("Budget Range Label")["budget"].count())*100).to_frame("% Passing Overall")

spendMathReadingOverallPass = spendPassMath.merge(spendPassReading, left_index=True, right_index=True).merge(spendPassOverall, left_index=True, right_index=True)
# spendMathReadingPass["OverallPassing"] = (spendMathReadingPass["% Passing Math"]+spendMathReadingPass["% Passing Reading"])/2

SpendingScores = spendMathAvg.merge(spendReadingAvg, left_index=True, right_index=True).merge(spendMathReadingOverallPass, left_index=True, right_index=True)
SpendingScores["MathAvg"] = SpendingScores["MathAvg"].map("{:.0f}".format)
SpendingScores["ReadingAvg"] = SpendingScores["ReadingAvg"].map("{:.0f}".format)
SpendingScores["% Passing Math"] = SpendingScores["% Passing Math"].map("{:.0f}".format)
SpendingScores["% Passing Reading"] = SpendingScores["% Passing Reading"].map("{:.0f}".format)
SpendingScores["% Passing Overall"] = SpendingScores["% Passing Overall"].map("{:.0f}".format)
SpendingScores

,MathAvg,ReadingAvg,% Passing Math,% Passing Reading,% Passing Overall
Budget Range Label,,,,,
\$575 - $599,83,84,94,97,91
\$600 - $624,84,84,94,96,90
\$625 - $649,78,81,71,84,60
\$650 - $674,77,81,66,81,54


##### Scores by School Size

In [10]:
sizeRange = [0,1000, 2000, 5000]
school["School Size Range"] = pd.cut(school["size"], np.array(sizeRange))
school["School Size Category"] = pd.cut(school["size"], np.array(sizeRange), labels = ["small (0 - 1000)","medium (1001 - 2000)","large (2001 - 5000)"], right=True)
# school
schoolStudents = school.merge(student, left_on="school", right_on="school")
# sizeStudents
# Average Math Score
sizeAvgMath = schoolStudents.groupby("School Size Category")["math_score"].mean().to_frame("AvgMath")
# Average Reading Score
sizeAvgReading = schoolStudents.groupby("School Size Category")["reading_score"].mean().to_frame("AvgReading")
# % Passing Math
sizePassMath = ((schoolStudents[schoolStudents["math_score"] >=70].groupby("School Size Category")["Student ID"].count()/schoolStudents.groupby("School Size Category")["Student ID"].count())*100).to_frame("% Passing Math")
# % Passing Reading
sizePassReading = ((schoolStudents[schoolStudents["reading_score"]>= 70].groupby("School Size Category")["reading_score"].count()/schoolStudents.groupby("School Size Category")["reading_score"].count())*100).to_frame("% Passing Reading")
# Overall Passing Rate (Average of the above two)
sizePassOverall = ((schoolStudents[(schoolStudents["math_score"]>= 70) & (schoolStudents["reading_score"]>= 70)].groupby("School Size Category")["Student ID"].count()/schoolStudents.groupby("School Size Category")["Student ID"].count())*100).to_frame("% Passing Overall")

sizeOverall = sizePassMath.merge(sizePassReading, left_index=True, right_index=True).merge(sizeAvgMath, left_index=True, right_index=True).merge(sizeAvgReading, left_index=True, right_index=True).merge(sizePassOverall, left_index=True, right_index=True)
# sizeOverall.columns = ["AvgReading","AvgMath","% Passing Math","% Passing Reading","% Passing Overall"]
sizeOverall["AvgReading"] = sizeOverall["AvgReading"].map("{:.0f}".format)
sizeOverall["AvgMath"] = sizeOverall["AvgMath"].map("{:.0f}".format)
sizeOverall["% Passing Math"] = sizeOverall["% Passing Math"].map("{:.0f}".format)
sizeOverall["% Passing Reading"] = sizeOverall["% Passing Reading"].map("{:.0f}".format)
sizeOverall["% Passing Overall"] = sizeOverall["% Passing Overall"].map("{:.0f}".format)
sizeOverall = sizeOverall.loc[:,["AvgReading","AvgMath","% Passing Math","% Passing Reading","% Passing Overall"]]
sizeOverall.sort_index(ascending=False)
# sizePassMath

,AvgReading,AvgMath,% Passing Math,% Passing Reading,% Passing Overall
School Size Category,,,,,
small (0 - 1000),84,84,94,96,90
medium (1001 - 2000),84,83,94,97,91
large (2001 - 5000),81,77,69,82,57


##### Scores by School Type 

In [11]:
typeAvgMath = schoolStudents.groupby("type")["math_score"].mean().to_frame("AvgMath")
typeAvgReading = schoolStudents.groupby("type")["reading_score"].mean().to_frame("AvgReading")
typePassMath = ((schoolStudents[schoolStudents["math_score"]>=70].groupby("type")["math_score"].count()/schoolStudents.groupby("type")["math_score"].count())*100).to_frame("% Passing Math")
typePassReading = ((schoolStudents[schoolStudents["reading_score"]>=70].groupby("type")["reading_score"].count()/schoolStudents.groupby("type")["reading_score"].count())*100).to_frame("% Passing Reading")
typePassBoth = ((schoolStudents[(schoolStudents["math_score"]>=70) & (schoolStudents["reading_score"]>=70)].groupby("type")["Student ID"].count()/schoolStudents.groupby("type")["Student ID"].count())*100).to_frame("% Passing Overall")

typePassOverall = typePassMath.merge(typePassReading, left_index=True, right_index=True).merge(typeAvgMath, left_index=True, right_index=True).merge(typeAvgReading,left_index=True, right_index=True).merge(typePassBoth,left_index=True, right_index=True)
# typePassOverall["Overall Passing"] = (typePassOverall["% Passing Math"]+typePassOverall["% Passing Reading"])/2
typePassOverall["AvgReading"] = typePassOverall["AvgReading"].map("{:.0f}".format)
typePassOverall["AvgMath"] = typePassOverall["AvgMath"].map("{:.0f}".format)
typePassOverall["% Passing Math"] = typePassOverall["% Passing Math"].map("{:.0f}".format)
typePassOverall["% Passing Reading"] = typePassOverall["% Passing Reading"].map("{:.0f}".format)
typePassOverall["% Passing Overall"] = typePassOverall["% Passing Overall"].map("{:.0f}".format)
typePassOverall = typePassOverall.loc[:,["AvgMath","AvgReading","% Passing Math","% Passing Reading","% Passing Overall"]]
typePassOverall

,AvgMath,AvgReading,% Passing Math,% Passing Reading,% Passing Overall
type,,,,,
Charter,83,84,94,97,91
District,77,81,67,81,54
